In [1]:
!pip install pandera ipytest

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 kB 1.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 3.5 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.8/316.8 kB 3.0 MB/s eta 0:00:00a 0:00:01


# Introduction

In [13]:
import pandas as pd
import pandera as pa

fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "price": [2, 1, 3, 4],
    }
)

fruits

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4


### 情境1
資料中只能特定水果、商店且價格只能低於 4 。  
為確保您的數據符合這些條件，手動檢查會耗費許多時間，
當資料量比較大時，可以利用 Pandera 做自動化檢查。  
* 用 DataFrameSchema 對整份資料建立多個測試
* 使用 Column 針對每個欄位建立測試
* 使用 Check 對欄位建立條件測試

In [3]:
available_fruits = ["apple", "banana", "orange"]
nearby_stores = ["Aldi", "Walmart"]

In [11]:
import pandera as pa
from pandera import Column, Check

available_fruits = ["apple", "banana", "orange"]
nearby_stores = ["Aldi", "Walmart"]
schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)), # 檢查欄位是否在特定類別水果
        "store": Column(str, Check.isin(nearby_stores)), # 檢查欄位是否在特定類別店家
        "price": Column(int, Check.less_than(4)), # 檢查價格是否小於 4
    }
)
schema.validate(fruits) # 做資料驗證

SchemaError: <Schema Column(name=price, type=DataType(int64))> failed element-wise validator 0:
<Check less_than: less_than(4)>
failure cases:
   index  failure_case
0      3             4

由 SchemaError 可以看到 index = 3 的資料有問題, 其 price = 4

In [6]:
schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(str, Check.isin(nearby_stores)),
        "price": Column(int, Check.less_than(5)),
    }
)
schema.validate(fruits)

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4


### Custom Check
* One Column Multiple Check
* Custom Check Using lambda

In [12]:
schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(str, Check.isin(nearby_stores)),
        "price": Column(
            int,
            [
                Check.less_than(5), # 檢查價格小於 5
                Check(lambda price: sum(price) < 20), # 檢查價格總和小於 20
            ],
        ),
    }
)
schema.validate(fruits)


,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4


# Schema Model
當測試較於複雜時，可以運用 dataclass 比使用 dict 看起來乾淨一點。

In [14]:
from pandera.typing import Series


class Schema(pa.SchemaModel):
    name: Series[str] = pa.Field(isin=available_fruits)
    store: Series[str] = pa.Field(isin=nearby_stores)
    price: Series[int] = pa.Field(le=5)

    @pa.check("price")
    def price_sum_lt_20(cls, price: Series[int]) -> Series[bool]:
        return sum(price) < 20


Schema.validate(fruits)

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4


# Validation Decorator 

## Check Input
針對 Input Data 測試

In [15]:
fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "price": [2, 1, 3, 4],
    }
)

schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(str, Check.isin(nearby_stores)),
        "price": Column(int, Check.less_than(5)),
    }
)


def get_total_price(fruits: pd.DataFrame, schema: pa.DataFrameSchema):
    validated = schema.validate(fruits)
    return validated["price"].sum()


get_total_price(fruits, schema)

10

In [16]:
def test_get_total_price():
    fruits = pd.DataFrame({'name': ['apple', 'banana'], 'store': ['Aldi', 'Walmart'], 'price': [1, 2]})
    
    schema = pa.DataFrameSchema(
        {
            "name": Column(str, Check.isin(available_fruits)),
            "store": Column(str, Check.isin(nearby_stores)),
            "price": Column(int, Check.less_than(5)),
        }
    )
    assert get_total_price(fruits, schema) == 3

由於上面兩個範例在做測試資料與函數會需要的參數包括 fruits, schema 比較多餘。  
可以運用 decorator 進行測試。

In [19]:
from pandera import check_input

fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "price": ["2", "1", "3", "4"],
    }
)


schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(str, Check.isin(nearby_stores)),
        "price": Column(int, Check.less_than(5)),
    }
)

@check_input(schema)
def get_total_price(fruits: pd.DataFrame):
    return fruits.price.sum()


get_total_price(fruits)

SchemaError: error in check_input decorator of function 'get_total_price': expected series 'price' to have type int64, got object

In [20]:
def test_get_total_price():
    fruits = pd.DataFrame({'name': ['apple', 'banana'], 'store': ['Aldi', 'Walmart'], 'price': [1, 2]})
    assert get_total_price(fruits) == 3

## Check Output

In [21]:
fruits_nearby = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "price": [2, 1, 3, 4],
    }
)

fruits_faraway = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Whole Foods", "Whole Foods", "Schnucks", "Schnucks"],
        "price": [3, 2, 4, 5],
    }
)

out_schema = pa.DataFrameSchema(
    {"store": Column(str, Check.isin(["Aldi", "Walmart", "Whole Foods", "Schnucks"]))}
)


@check_output(out_schema)
def combine_fruits(fruits_nearby: pd.DataFrame, fruits_faraway: pd.DataFrame):
    fruits = pd.concat([fruits_nearby, fruits_faraway])
    return fruits


combine_fruits(fruits_nearby, fruits_faraway)

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4
0,apple,Whole Foods,3
1,banana,Whole Foods,2
2,apple,Schnucks,4
3,orange,Schnucks,5


## Check Both

In [22]:
# 只驗證資料型態
in_schema = pa.DataFrameSchema({"store": Column(str)})

# 驗證轉換後資料
out_schema = pa.DataFrameSchema(
    {"store": Column(str, Check.isin(["Aldi", "Walmart", "Whole Foods", "Schnucks"]))}
)


@check_io(fruits_nearby=in_schema, fruits_faraway=in_schema, out=out_schema)
def combine_fruits(fruits_nearby: pd.DataFrame, fruits_faraway: pd.DataFrame):
    fruits = pd.concat([fruits_nearby, fruits_faraway])
    return fruits


combine_fruits(fruits_nearby, fruits_faraway)

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4
0,apple,Whole Foods,3
1,banana,Whole Foods,2
2,apple,Schnucks,4
3,orange,Schnucks,5


# Other Arguments for Column Validation

## Deal with Null Values
* nullable = True 允許遺失值

In [7]:
import numpy as np

fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", np.nan],
        "price": [2, 1, 3, 4],
    }
)

fruits

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,NaN,4


In [19]:
schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(
            str,
            Check.isin(nearby_stores),
            nullable=True,
        ),
        "price": Column(int, Check.less_than(5)),
    }
)
schema.validate(fruits)


,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,NaN,4


## Deal with Duplicates
* unique = True 只能有唯一值

In [26]:
schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(
            str,
            Check.isin(nearby_stores),
            nullable=True,
            unique=True
        ),
        "price": Column(int, Check.less_than(5)),
    }
)
schema.validate(fruits)


SchemaError: series 'store' contains duplicate values:
2    Walmart
Name: store, dtype: object

## Convert Data Types
* coerce=Ture, 強制轉換資料型別

In [29]:
fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "price": [2, 1, 3, 4],
    }
)

schema = pa.DataFrameSchema({"price": Column(str, coerce=True)})
validated = schema.validate(fruits)
validated.dtypes

name     object
store    object
price    object
dtype: object

## Patern Matching
* regex = Ture 與 多資料欄位

In [ ]:
favorite_stores = ["Aldi", "Walmart", "Whole Foods", "Schnucks"]

fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store_nearby": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "store_far": ["Whole Foods", "Schnucks", "Whole Foods", "Schnucks"],
    }
)

schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store_+": Column(
            str,
            Check.isin(favorite_stores),
            regex=True,
        ),
    }
)
schema.validate(fruits)


In [32]:
favorite_stores = ["Aldi", "Walmart", "WF", "Schnucks"]

fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store_nearby": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "store_far": ["WF", "Schnucks", "WF", "Schnucks"],
    }
)

schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store_*": Column(
            str,
            Check.isin(favorite_stores),
            regex=True
        ),
    }
)
schema.validate(fruits)


,name,store_nearby,store_far
0,apple,Aldi,WF
1,banana,Walmart,Schnucks
2,apple,Walmart,WF
3,orange,Aldi,Schnucks


# Export and Load From a YAML file

## Export

In [49]:
yaml_schema = schema.to_yaml()
print(yaml_schema)

schema_type: dataframe
version: 0.7.0
columns:
  name:
    dtype: str
    nullable: false
    checks:
      isin:
      - apple
      - banana
      - orange
    allow_duplicates: true
    coerce: false
    required: true
    regex: false
  store:
    dtype: str
    nullable: true
    checks:
      isin:
      - Aldi
      - Walmart
    allow_duplicates: false
    coerce: false
    required: true
    regex: false
  price:
    dtype: int64
    nullable: false
    checks:
      less_than: 5
    allow_duplicates: true
    coerce: false
    required: true
    regex: false
checks: null
index: null
coerce: false
strict: false



<IPython.core.display.Javascript object>

In [53]:
from pathlib import Path

f = Path("schema.yml")
f.touch()
f.write_text(yaml_schema)

628

<IPython.core.display.Javascript object>

## Load

In [56]:
with f.open() as file:
    yaml_schema = file.read()

<IPython.core.display.Javascript object>

In [57]:
schema = pa.io.from_yaml(yaml_schema)
schema

<Schema DataFrameSchema(columns={'name': <Schema Column(name=name, type=DataType(str))>, 'store': <Schema Column(name=store, type=DataType(str))>, 'price': <Schema Column(name=price, type=DataType(int64))>}, checks=[], index=None, coerce=False, dtype=None,strict=False,name=None,ordered=False)>

<IPython.core.display.Javascript object>